# OKCupid User Age Prediction Project

### **Introduction**

This project analyzes data from the online dating application OKCupid. In recent years, there has been a massive rise in the usage of dating apps to find love. Many of these apps use sophisticated data science techniques to recommend possible matches to users and to optimize the user experience. These apps give us access to a wealth of information that we've never had before about how different people experience romance. The goal of this project is to scope, prep, analyze, and create a machine learning model to solve a key question about the user base.

---

### **Scoping**

It's beneficial to create a project scope whenever a new project is being started. Below are four sections to help guide the project process and progress. The first section is the project goals, a section to define the high-level objectives and set the intentions for this project. The next section is the data; luckily, in this project, data is already provided but still needs to be checked if project goals can be met with the available data. Thirdly, the analysis will have to be thought through, which includes the methods and aligning the question(s) with the project goals. Lastly, evaluation will help build conclusions and findings from the analysis.

---

### **Project Goals**

In this project, the goal is to utilize the skills learned through Codecademy and apply machine learning techniques to a data set. The primary research question that will be answered is **whether an OkCupid user's age can be accurately predicted using other variables from their profile**, with a special focus on their essay responses, education level, and lifestyle choices.

This project is important as it provides value in multiple ways. For OkCupid, it could help identify profiles where age may be misrepresented, improving user trust and safety. Furthermore, it provides deep insight into the correlations between age and how users choose to describe themselves and their life stages, which can be used to further refine the user experience. This project will involve building a **regression model**, demonstrating a versatile and highly in-demand data science skill.

---

### **Data**

The project has one data set provided by Codecademy called `profiles.csv`. In the data, each row represents an OkCupid user and the columns are the responses to their user profiles which include multi-choice and short answer questions. The target variable for this project will be the `age` column, a numerical value. Features will be drawn from the other columns, including categorical data like `education` and `drinks`, and unstructured text data from the `essay` columns.

---

### **Analysis**

This solution will use descriptive statistics and data visualization to find key figures in understanding the distribution of user ages and the relationships between age and other variables. Since the goal of the project is to predict a continuous numerical value (`age`), **regression algorithms** from the supervised learning family of machine learning models will be implemented.

A significant part of the analysis will involve **feature engineering**, particularly processing the natural language in the `essay` columns to create meaningful numerical features. Techniques like TF-IDF or word counts may be used to quantify writing styles. The performance of various regression models, such as Linear Regression, Random Forest Regressor, and Gradient Boosting, will be compared to find the best-performing solution.

---

### **Evaluation**

The project will conclude with the evaluation of the final machine learning model on a validation data set. Because this is a regression task, the model's performance will be measured using metrics appropriate for continuous data. The output of the predictions will be checked using metrics such as:
* **Mean Absolute Error (MAE):** Measures the average magnitude of the errors in a set of predictions, without considering their direction.
* **Root Mean Squared Error (RMSE):** The standard deviation of the prediction errors, giving a sense of how concentrated the data is around the line of best fit. It penalizes larger errors more than MAE.
* **R-squared ($R^2$):** The coefficient of determination, which represents the proportion of the variance in the `age` variable that is predictable from the features.

In [ ]:
## Import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Loading Data
To begin the analysis, the profiles.csv dataset is loaded into a pandas DataFrame named profiles. The .head() method is then used to preview the first few rows, providing an initial look at the data's structure and confirming a successful import.

In [ ]:
profiles = pd.read_csv('profiles.csv', encoding='utf-8')
profiles.head(5)

In [ ]:
profiles.last_online.head()

#### Data Characteristics

The `profiles` DataFrame contains 59,946 rows and 31 columns, providing a substantial dataset for this analysis. This volume of data is considered more than sufficient for machine learning, as it far exceeds the common rule of thumb of having at least 10 times as many data points as features (59,946 >> 10 * 31).

In [ ]:
profiles.info()

## Data Dictionary: `profiles.csv`

| Column Name | Original Type | Refined Data Type | Rationale |
| :--- | :--- | :--- | :--- |
| **Demographics** | | | |
| `sex` | Categorical | **Categorical (Binary)** | This is a categorical variable with only two options (m/f). |
| `age` | Numerical | **Numerical (Integer)** | Age is a whole number. Matches the `int64` dtype. |
| `orientation` | Categorical | **Categorical (Nominal)** | The categories have no intrinsic order. |
| `ethnicity` | Categorical | **Categorical (Multi-value)** | A user can list multiple ethnicities, often stored as a comma-separated string. |
| `location` | Categorical | **Geospatial/String** | More specific than generic 'Categorical'. Stored as a string (`object`). |
| **Physical & Status** | | | |
| `body_type` | Categorical | **Categorical (Nominal)** | Body type categories typically do not have a mathematical order. |
| `height` | Numerical | **Numerical (Float)** | Height can include decimal values. Matches the `float64` dtype. |
| `income` | Numerical | **Numerical (Integer)** | Annual income is represented as a whole number. Matches the `int64` dtype. |
| `status` | Categorical | **Categorical (Nominal)** | Relationship statuses (e.g., single, married) have no inherent order. |
| **Education & Work** | | | |
| `education` | Categorical | **Categorical (Ordinal)** | There is a clear order in educational attainment (e.g., high school < bachelor's). |
| `job` | Categorical | **Categorical (Nominal)** | Job categories generally have no intrinsic ranking. |
| **Lifestyle & Family** | | | |
| `diet` | Categorical | **Categorical (Nominal)** | Dietary habits are distinct categories without a clear mathematical order. |
| `drinks` | Categorical | **Categorical (Ordinal)** | Alcohol consumption is often ranked (e.g., not at all < socially < often). |
| `smokes` | Categorical | **Categorical (Ordinal)** | Smoking habits have a natural order (e.g., no < sometimes < yes). |
| `drugs` | Categorical | **Categorical (Ordinal)** | Drug use habits also have a ranked order (e.g., never < sometimes). |
| `pets` | Categorical | **Categorical (Multi-value)** | A user can have preferences for multiple types of pets. |
| `offspring` | Categorical | **Categorical (Ordinal)** | There can be an order (e.g., doesn't have < has kids). |
| `religion` | Categorical | **Categorical (Nominal)** | Religious beliefs are nominal, though the 'intensity' part could be ordinal. |
| `sign` | Categorical | **Categorical (Nominal)** | Astrological signs have no order, though the 'intensity' part could be ordinal. |
| **Communication & Essays** | | | |
| `speaks` | Categorical | **Categorical (Multi-value)** | A user can speak multiple languages. |
| `last_online` | Datetime | **Datetime** | The `info()` shows this as `object`, but it represents a specific point in time and should be converted. |
| `essay0`-`essay9` | Text | **Text (String)** | Correctly identified as open-ended text. The `object` dtype in pandas is appropriate here. |

## Cleaning the Data


In [ ]:
# A complete list of all categorical columns to be imputed
all_categorical_cols = [
    'offspring', 'religion', 'pets', 'drugs', 'smokes', 'diet', 'body_type', 'job',
    'drinks', 'education', 'ethnicity', 'sign', 'speaks'
]

# List of all essay columns
all_essay_cols = [f"essay{i}" for i in range(10)]

# Impute all categorical columns
for col in all_categorical_cols:
    profiles[col].fillna('not specified', inplace=True)

# Impute all essay columns
for col in all_essay_cols:
    profiles[col].fillna('', inplace=True)

# Impute height with the median
profiles['height'].fillna(profiles['height'].median(), inplace=True)

# This will also return True if there are no null values.
not profiles.isna().any().any()

## Analyzing the Target Variable `age`

In [ ]:
# Histogram of the 'age' column
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))
sns.histplot(data=profiles, x='age', bins=40, kde=True)
plt.title('Distribution of User Ages')
plt.xlabel('Age')
plt.ylabel('Number of Users')
plt.savefig('Distribution_of_User_Ages.png')
plt.show()

**Peak Age Group**: The vast majority of users are in their mid-to-late 20s and early 30s. The single highest concentration of users appears to be around age 26.

**Right-Skewed Distribution**: The data is heavily right-skewed, meaning there is a long tail of users extending into older age groups. After peaking in the late 20s, the number of users per age group steadily declines.

**Potential Outliers**: There is a very long tail with users listed at improbable ages for a dating app, such as those over 100. These are almost certainly outliers or users entering joke data.

**Outlier Handling is Necessary**: Before training our model we will filter out the extreme age outliers (e.g., anyone over a reasonable age like 80). These data points will negatively impact our model's performance and evaluation metrics.

**Model Performance May Vary by Age**: The model will have a large amount of data to learn from for users in their 20s and 30s, so it will likely be most accurate in that range. It may be less accurate when predicting the age of older users simply because there is less data available for it to learn from.

In [ ]:
# --- Outlier Handling for Age ---

# First, let's see how many rows we have before filtering
print(f"Original number of profiles: {len(profiles)}")

# Ages above this are rare and may be invalid data
AGE_UPPER_LIMIT = 70

# Create a new DataFrame that only includes users within the age limit
profiles_filtered = profiles[profiles['age'] <= AGE_UPPER_LIMIT].copy()

# Report how many outliers were removed
print(f"Number of profiles after removing users older than {AGE_UPPER_LIMIT}: {len(profiles_filtered)}")
print(f"Number of outliers removed: {len(profiles) - len(profiles_filtered)}")

In [ ]:
# Create a box plot to see age distribution by education level
plt.figure(figsize=(14, 8))
sns.boxplot(data=profiles_filtered, x='education', y='age')
plt.title('Age Distribution by Education Level')
plt.xlabel('Education Level')
plt.ylabel('Age')
plt.xticks(rotation=45, ha='right') 
plt.tight_layout()
plt.savefig('Age_Distribution_by_Education_Level.png')
plt.show()

### Key Observations

* **Clear Age Progression:** There is a distinct upward trend in age as the level of education increases. Users with a high school education are generally the youngest, followed by those in college, then masters programs, and finally Ph.D., law, and med school graduates, who are the oldest.

* **"Working On" vs. "Graduated":** For every level of schooling (college, masters, etc.), the "working on" category consistently shows a lower median age than the "graduated from" category. This is a powerful and logical signal.

* **Distinct Groupings:** The plot clearly separates users into life-stage groups. For example, the median age for those "working on college/university" is in the low 20s, while the median for those who "graduated from law school" is in the mid-30s.

---

In [ ]:
# Select only the numerical columns for the correlation matrix
numerical_cols = profiles_filtered.select_dtypes(include=['int64', 'float64'])

# Calculate the correlation matrix
correlation_matrix = numerical_cols.corr()

# Create the heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numerical Features')
plt.savefig('Correlation_Matrix_of_Numerical_Features.png')
plt.show()

### Key Observations 

* **No Correlation with `height` and `income`:** The correlation between `age` and `height` (-0.02) and `age` and `income` (-0.00) is effectively zero. This indicates there is no linear trend between these features. The lack of correlation with income is particularly interesting and may be due to how users reported their income.

* **Weakest Signal from `essay_length`:** The correlation between `age` and `essay_length` is 0.08. While this is the highest correlation with `age` on the chart, it is still very weak. It suggests only a slight tendency for older users to write more, but the relationship is not strong.



These simple numerical features, on their own, will not be strong predictors of `age`.

This reinforces the importance of using **categorical features** (like `education`) and developing more advanced **NLP features** from the essay text.

In [ ]:
# Combine all essay columns into a single text block using the filtered DataFrame
essay_cols = [f'essay{i}' for i in range(10)]
profiles_filtered['all_essays'] = profiles_filtered[essay_cols].apply(lambda x: ' '.join(x), axis=1)

# Calculate the length of the combined essays
profiles_filtered['essay_length'] = profiles_filtered['all_essays'].str.len()

# Create a scatter plot of essay length vs. age from the filtered data
# Use a sample to avoid overplotting
sample_df = profiles_filtered.sample(n=5000, random_state=42)
plt.figure(figsize=(12, 7))
sns.scatterplot(data=sample_df, x='age', y='essay_length', alpha=0.5)
plt.title('Essay Length vs. Age (Outliers Removed)')
plt.xlabel('Age')
plt.ylabel('Total Essay Character Length')
plt.savefig('Essay_Length_vs_Age.png')
plt.show()

### Key Observations 

* **No Clear Trend:** The points form a large, dense cloud rather than a distinct line. This confirms the very weak correlation (0.08) we saw in the heatmap. We can't predict a user's age based on how much they write.
* **Highest Density in the 20-40 Age Range:** The densest area of the plot is for users between 20 and 40 years old with essay lengths under 5,000 characters. This is expected, as this is the largest user demographic.
* **Most Users Write Moderately:** Regardless of age, the vast majority of users write less than 5,000 characters. Very long essays are rare across all age groups.

This Chart shows that **`essay_length` by itself is a weak feature** for predicting age.

To get predictive power from the essays, we will need to move beyond simple metrics like length and analyze the **actual content** of the text. The vocabulary, topics, and writing style are much more likely to change with age than the sheer volume of text. This reinfoces the possiblity of using more advanced NLP techniques (like TF-IDF or word embeddings) in the next stage of the project.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to classify offspring status into simpler categories
def clean_offspring(status):
    if 'has kids' in status or 'has a kid' in status:
        return 'has kids'
    elif "doesn't have kids" in status or 'wants kids' in status:
        return "doesn't have kids"
    else:
        return 'not specified'

# Apply the function to create the new cleaned column
profiles_filtered['offspring_cleaned'] = profiles_filtered['offspring'].apply(clean_offspring)

# Set the style for the plots
sns.set_style("whitegrid")

# Create a box plot to see age distribution by offspring status
plt.figure(figsize=(14, 8))
sns.boxplot(data=profiles_filtered, x='offspring_cleaned', y='age')
plt.title('Age Distribution by Offspring Status')
plt.xlabel('Offspring Status')
plt.ylabel('Age')
plt.xticks(rotation=45, ha='right') # Rotate labels for better readability
plt.tight_layout()
plt.savefig('Age_Distribution_by_Offspring_Status.png')
plt.show()

### Key Observations

* **Two Distinct Groups:** The visualization clearly separates the user base into two demographic groups: users who have kids and users who do not.
* **Strong Separation:** There's a dramatic difference in age between these groups. The median age for users with kids (mid-40s) is significantly higher than for those without kids (late 20s). The boxes for these groups have almost no overlap.
* **"Not Specified" Insight:** The "not specified" category has an age distribution nearly identical to the "doesn't have kids" category. This confirms that users who don't fill out this field are overwhelmingly in the younger, non-parent demographic.

This confirms that `offspring_cleaned` column is an extremely strong and reliable feature for predicting `age`.

## Feature Engineering

This was the most critical phase of the project, where raw data was transformed into numerical features suitable for a machine learning model. A multi-step approach was used to handle the variety of data types present in the dataset.

* **Text Simplification:** The 10 separate `essay` columns were combined into a single text block for each user. From this, simple metrics like total character count (`essay_length`) and word count (`word_count`) were calculated.
* **Datetime and Location Features:** The `last_online` string was converted into a proper datetime object to extract numerical features like the year, month, and day of the week. Similarly, the `location` string was parsed to extract the user's `state`, creating a useful categorical feature.
* **Categorical Simplification:** To reduce noise, messy categorical columns like `offspring`, `religion`, and `sign` were cleaned by custom functions to extract the core category (e.g., `"gemini"` from `"gemini and it doesn't matter"`).
* **Multi-Label Feature Creation:** For columns like `ethnicity` and `speaks` where users could list multiple values (e.g., `'asian, white'`), a specialized `get_dummies` approach was used to create binary columns for each possible entry. This correctly captures users belonging to multiple groups.
* **Final Encoding:** After all the cleaning and creation steps, the final set of categorical features was one-hot encoded. Simultaneously, **TF-IDF (Term Frequency-Inverse Document Frequency)** was applied to the combined essays to convert the text into a 500-column numerical matrix based on word importance.
* **Assembly:** All engineered features—simple metrics, one-hot encoded columns, and the TF-IDF matrix—were combined into a final feature set containing **1002 features** ready for modeling.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

# --- 1. Load and Initial Cleaning ---
profiles = pd.read_csv('profiles.csv')
profiles_filtered = profiles[profiles['age'] <= 65].copy()

# --- 2. Impute All Missing Data ---
essay_cols = [f"essay{i}" for i in range(10)]
fill_values = {
    'body_type': 'not specified', 'diet': 'not specified', 'drinks': 'not specified',
    'drugs': 'not specified', 'education': 'not specified', 'ethnicity': 'not specified',
    'job': 'not specified', 'offspring': 'not specified', 'pets': 'not specified',
    'religion': 'not specified', 'sign': 'not specified', 'smokes': 'not specified',
    'speaks': 'not specified', 'height': profiles_filtered['height'].median()
}
for col in essay_cols:
    fill_values[col] = ''
profiles_filtered.fillna(value=fill_values, inplace=True)


# --- 3. Feature Engineering ---

# A) Combine text and engineer simple features
profiles_filtered['all_essays'] = profiles_filtered[essay_cols].apply(lambda x: ' '.join(x), axis=1)
profiles_filtered['essay_length'] = profiles_filtered['all_essays'].str.len()
profiles_filtered['word_count'] = profiles_filtered['all_essays'].str.split().str.len()

# B) Engineer location, datetime, and cleaned categorical features
profiles_filtered[['city', 'state']] = profiles_filtered['location'].str.split(',', n=1, expand=True)
profiles_filtered['state'] = profiles_filtered['state'].str.strip()
profiles_filtered['last_online_dt'] = pd.to_datetime(profiles_filtered['last_online'], format='%Y-%m-%d-%H-%M')
profiles_filtered['last_online_year'] = profiles_filtered['last_online_dt'].dt.year
profiles_filtered['last_online_month'] = profiles_filtered['last_online_dt'].dt.month
profiles_filtered['last_online_dayofweek'] = profiles_filtered['last_online_dt'].dt.dayofweek

def clean_offspring(status):
    if 'has kids' in status or 'has a kid' in status: return 'has kids'
    elif "doesn't have kids" in status or 'wants kids' in status: return "doesn't have kids"
    else: return 'not specified'
profiles_filtered['offspring_cleaned'] = profiles_filtered['offspring'].apply(clean_offspring)

def clean_religion(religion_str): return religion_str.split(" ")[0]
profiles_filtered['religion_cleaned'] = profiles_filtered['religion'].apply(clean_religion)

def clean_sign(sign_str): return sign_str.split(" ")[0]
profiles_filtered['sign_cleaned'] = profiles_filtered['sign'].apply(clean_sign)

# C) Create dummy columns for multi-label text features (ethnicity, speaks)
ethnicity_dummies = profiles_filtered['ethnicity'].str.get_dummies(sep=', ')
speaks_dummies = profiles_filtered['speaks'].str.get_dummies(sep=', ') # Added for 'speaks'
profiles_with_dummies = pd.concat([profiles_filtered, ethnicity_dummies, speaks_dummies], axis=1)

# D) One-Hot Encode the remaining categorical features
features_to_encode = [
    'body_type', 'diet', 'drinks', 'drugs', 'education', 'job', 'orientation',
    'pets', 'smokes', 'status', 'sex', 'state',
    'offspring_cleaned', 'religion_cleaned', 'sign_cleaned'
]
profiles_encoded = pd.get_dummies(profiles_with_dummies, columns=features_to_encode, drop_first=True)

# E) Create advanced text features with TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
tfidf_features = tfidf_vectorizer.fit_transform(profiles_encoded['all_essays'])


# --- 4. Assemble Final Data and Split for Modeling ---

# A) Define the target variable
y = profiles_encoded['age']

# B) Select all final feature columns
# --- UPDATED: Added 'speaks' to the drop list ---
cols_to_drop_for_X = [
    'age', 'all_essays', 'offspring', 'religion', 'sign', 'ethnicity', 'speaks', # Added 'speaks'
    'location', 'city', 'last_online', 'last_online_dt'
] + essay_cols
numeric_and_ohe_features = profiles_encoded.drop(columns=cols_to_drop_for_X)

# C) Combine all features into the final matrix X
X = hstack([numeric_and_ohe_features.astype(float).values, tfidf_features])

# D) Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 5. Verification ---
print("Feature engineering and data preparation complete.")
print("Final training feature matrix shape:", X_train.shape)
print("Final testing feature matrix shape:", X_test.shape)

### Training a Baseline Moded (Ridge Regression)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Initialize the Model
# We use a standard alpha (regularization strength).
model = Ridge(alpha=1.0)

# 2. Train the Model
# The model learns the relationships between features and the age from the training data.
print("Training the model...")
model.fit(X_train, y_train)
print("Training complete.")

# 3. Make Predictions on the Test Data
# The model now predicts the age for the users in the test set.
y_pred = model.predict(X_test)

# 4. Evaluate the Model's Performance
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- Model Evaluation ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

## Analysis of Model Performance

This is a **solid baseline result**. The model can explain about half of the variation in user age, and its predictions are, on average, off by just under 5 years.

### Performance Breakdown

* **R-squared (R²): 0.52:** Features explain **52% of the variance** in user ages. This indicates that the features engineered—especially from `education`, `offspring`, and the essays—have significant predictive power.
* **Mean Absolute Error (MAE): 4.84 years:** On average, our model's prediction of a user's age is off by about **4.84 years**. Given the age range of 18-65, this is a reasonable starting point.

---
### How to Improve the Model

1.  **More Powerful Models:** Tree-based models are excellent for this type of problem. The next logical step would be to train a **`RandomForestRegressor`** or a **`GradientBoostingRegressor`** from Scikit-learn. These models can often uncover more complex patterns than a linear model like Ridge.
2.  **Hyperparameter Tuning:** You can fine-tune your model's settings (hyperparameters) using `GridSearchCV` or `RandomizedSearchCV` to squeeze out better performance.
3.  **Advanced Feature Engineering:** We could experiment with more advanced NLP techniques like **word embeddings** (e.g., Word2Vec) instead of TF-IDF to better capture the meaning of the essay text.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Initialize the Random Forest Model
# n_estimators is the number of trees in the forest.
# n_jobs=-1 uses all available CPU cores to speed up training.
# random_state=42 ensures the results are reproducible.
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# 2. Train the Model
# This will take longer than the Ridge model.
print("Training the Random Forest model...")
rf_model.fit(X_train, y_train)
print("Training complete.")

# 3. Make Predictions on the Test Data
y_pred_rf = rf_model.predict(X_test)

# 4. Evaluate the Model's Performance
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("\n--- Random Forest Model Evaluation ---")
print(f"Mean Absolute Error (MAE): {mae_rf:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.2f}")
print(f"R-squared (R²): {r2_rf:.2f}")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define a grid of parameters to search
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Randomized Search
# It will try 10 different combinations from the grid above
rf_random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_dist, 
    n_iter=10, 
    cv=3, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

# Fit the search to find the best model
print("Performing randomized search for best hyperparameters...")
rf_random_search.fit(X_train, y_train)

# You can then use rf_random_search.best_estimator_ as your new, tuned model
print(f"Best parameters found: {rf_random_search.best_params_}")

In [ ]:
# Get the feature importances from the trained model
importances = rf_model.feature_importances_
feature_names = numeric_and_ohe_features.columns.tolist() + tfidf_vectorizer.get_feature_names_out().tolist()

# Create a DataFrame for easy sorting and plotting
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# Plot the top 20 most important features
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(20))
plt.title('Top 20 Most Important Features')
plt.savefig('Distribution_of_User_Ages.png')
plt.show()

## Final Model Analysis and Conclusion

### Feature Importance Insights

The feature importance plot is a fantastic validation the EDA.

* **Top Predictors Confirmed:** The model found that **`offspring_cleaned_has kids`** is by far the most important feature for predicting age. This, along with the high importance of education-related features (`education_working on college/university`, `job_student`, etc.), confirms that **key life-stage indicators are the strongest predictors**.
* **Logical Model:** This result shows that our model is learning logical, real-world patterns. It's not relying on obscure signals but on the features we hypothesized would be most impactful.
* **Text Features:** While many individual text features from the essays appear in the top 20, they have much lower importance scores than the top categorical features. This suggests that while text is helpful, the demographic and life-stage data provides the core predictive power.

### Hyperparameter Tuning and Final Model Choice

The randomized search for the Random Forest model settled on parameters that are very close to the default settings (`'max_depth': None`, etc.). This means that even after searching, the complex Random Forest model couldn't find a configuration that significantly outperformed its baseline, which was already weaker than the Ridge model.

The "winning" model is the **Ridge Regression model**.

* **Final R-squared (R²): 0.52**
* **Final Mean Absolute Error (MAE): 4.84 years**

### Project Summary 

We have prepared a complex, real-world dataset, engineered dozens of features from categorical and text data, and built and evaluated multiple models.

The final result is a model that can predict a user's age with an average error of **~4.8 years**, explaining about 52% of the variance in the data. The analysis concludes that for this specific dataset, a simpler, regularized linear model (Ridge) was more effective than a more complex, tree-based model (Random Forest).

#### Next Steps 

Fine-tuning the model further will likely yield only minor improvements and is far less valuable. Therfore we will create a simple web application using a tool like Streamlit or Flask. The app could have a simple form where a user enters some hypothetical data (e.g., selects their education, offspring status, and writes a short "about me" summary), and the model predicts their age.